# PRACTICA GUIADA 2: Pivot Tables

* Una pivot table esuna operación similar a un `GroupBy` que suele ser común en planillas de cálculo y otros programas que operan con datos tabulares.
* Una pivot table toma una o varias columnas como input y agrupa las entradas en una tabla bidimensional que provee un resumen (generalmente, una agregación de los datos).

## Usando Pivot Tables

* Usaremos el dataset de pasajeros en el Titanic.
* El mismo contiene información sociodemográfica acerca de los pasajeros del barco (incluyendo, sexo, edad, clase de embarque, etc.)

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
titanic = sns.load_dataset('titanic')

In [3]:
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


## Pivot Tables a mano

* Comencemos agrupando de acuerdo al género, si sobrevieron o no, etc.
* Esto podríamos hacerlo con un ``GroupBy``; veamos, por ejemplo, la proporción de sobrevivientes según sexo:

In [4]:
titanic.groupby('sex')[['survived']].mean()

,survived
sex,
female,0.742038
male,0.188908


* Parece que 3 de cada 4 mujeres sobrevivieron, mientras que esta relación es notablemente menor entre los hombres (1 de cada 5)
* Veamos qué pasa si analizamos la sobrevivencia por sexo y clase:
    - hacemos un group por clase y género, seleccionamos los sobrevimientos, aplicamos una agregación (media) y combinamos los grupos resultantes y luego, "unstackeamos" el índice jerárquico:

In [5]:
titanic.groupby(['sex', 'class'])['survived'].aggregate('mean').unstack()

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


* Esto da una idea mejor de cómo el género y la clase afectan las probabilidades de sobreviviencia. El problema es que el código empieza a parece un poco desorndeado y poco sencillo para leer.
* Este tipo de ``GroupBy`` es muy común en Pandas, por eso se incluyó un método ``pivot_table`` que maneja fácilmente este tipo de agregaciones multidimensionales.

## Sintaxis de las Pivot Table

* Veamos un equivalente de la operación previa usando el método ``pivot_table``:

In [6]:
titanic.pivot_table('survived', index='sex', columns='class')

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


* Es claramente más legible que ``groupby`` anterior.
* Como era esperable, había mayores probabildades (tanto entre hombres como mujeres) de sobrevivir si la persona provenía de clase alta.
* Las mujeres de la primera clase sobrevivieron casi en su totalidad (seguramente, por aquello de "las mujeres y los niños -con plata- primero").

In [41]:
titanic.pivot_table(index=['sex', 'class'])

adult_male        age     alone        fare     parch  pclass  \
sex    class                                                                   
female First     0.000000  34.611765  0.361702  106.125798  0.457447     1.0   
       Second    0.000000  28.722973  0.421053   21.970121  0.605263     2.0   
       Third     0.000000  21.750000  0.416667   16.118810  0.798611     3.0   
male   First     0.975410  41.281386  0.614754   67.226127  0.278689     1.0   
       Second    0.916667  30.740707  0.666667   19.741782  0.222222     2.0   
       Third     0.919308  26.507589  0.760807   12.661633  0.224784     3.0   

                  sibsp  survived  
sex    class                       
female First   0.553191  0.968085  
       Second  0.486842  0.921053  
       Third   0.895833  0.500000  
male   First   0.311475  0.368852  
       Second  0.342593  0.157407  
       Third   0.498559  0.135447

### Multi-level pivot tables

* Igual que en ``GroupBy``, la operación de grouping  puede ser especificada con múltiples niveles.
* Por ejemplo, podríamos estar interesados en usar grupos de edad como tercera dimensión
* Generamos bins de la variable 'age' usando ``pd.cut``:

In [9]:
# genera rangos setarios 
age = pd.cut(titanic['age'], [0, 18, 80])
titanic.pivot_table('survived', ['sex', age], 'class', dropna = False)

class               First    Second     Third
sex    age                                   
female (18, 80]  0.909091  1.000000  0.511628
       NaN       0.972973  0.900000  0.423729
male   (18, 80]  0.800000  0.600000  0.215686
       NaN       0.375000  0.071429  0.133663

* Podemos usar la misma estrategia cuando trabajamos con las columnas: agreguemos información acerca de la tarifa (fare) pagada, usando ``pd.qcut`` para computar cuantiles de forma automática:

In [22]:
fare = pd.qcut(titanic['fare'], 2)
titanic.pivot_table('survived', ['sex', age], [fare, 'class'])

fare            (-0.001, 14.454]                     (14.454, 512.329]  \
class                      First    Second     Third             First   
sex    age                                                               
female (18, 80]              NaN  1.000000  0.714286          0.909091   
       NaN                   NaN  0.880000  0.444444          0.972973   
male   (18, 80]              NaN  0.000000  0.260870          0.800000   
       NaN                   0.0  0.098039  0.125000          0.391304   

fare                                 
class              Second     Third  
sex    age                           
female (18, 80]  1.000000  0.318182  
       NaN       0.914286  0.391304  
male   (18, 80]  0.818182  0.178571  
       NaN       0.030303  0.192308

### Opcionales adicionales en Pivot Tables

* Otros parámetros del método ``pivot_table`` son los siguientes:

```python
# call signature as of Pandas 0.18
DataFrame.pivot_table(data, values=None, index=None, columns=None,
                      aggfunc='mean', fill_value=None, margins=False,
                      dropna=True, margins_name='All')
```

* Vimos ejemplos de los primeros tres argumentos.
* ``fill_value`` y ``dropna``, se vinculan con la existencia de datos faltantes y son una forma relativamente simple de lidiar con ellos (volveremos sobre estos ejemplos más adelante).

* ``aggfunc`` controla el tipo de agregación que es aplicado (por defecto es una media)
* Al igual que con ``GroupBy``, la especificación de la operación de agregación tiene muchas opciones relativamete comunes (``'sum'``, ``'mean'``, ``'count'``, ``'min'``, ``'max'``, etc.) o bien alguna función que implementa una agregación (e.g., ``np.sum()``, ``min()``, ``sum()``, etc.).
* Además, puede expecificarse un diccionario que mapee columnas con operaciones

In [13]:
titanic.pivot_table(index='sex', columns='class',
                    aggfunc={'survived':sum, 'fare':'mean'})

fare                       survived             
class        First     Second      Third    First Second Third
sex                                                           
female  106.125798  21.970121  16.118810       91     70    72
male     67.226127  19.741782  12.661633       45     17    47

* En este caso, se omitión la especificación de ``values``: al especificar un mapeo para ``aggfunc``, `values` se determina automáticamente.

* A veces es útil computar totales a lo largo de cada grupo: podemos hacerlo usando ``margins``:

In [23]:
titanic.pivot_table('survived', index='sex', columns='class', margins=True)

class,First,Second,Third,All
sex,,,,
female,0.968085,0.921053,0.500000,0.742038
male,0.368852,0.157407,0.135447,0.188908
All,0.629630,0.472826,0.242363,0.383838


* La etiqueta del margen puede ser especificada con ``margins_name``, por defecto es ``"All"``.